In [9]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


### Step1. Import mnist dataset
- data set image size: 28*28

In [10]:
data = input_data.read_data_sets('data/MNIST/', one_hot=True)

Extracting data/MNIST/train-images-idx3-ubyte.gz
Extracting data/MNIST/train-labels-idx1-ubyte.gz
Extracting data/MNIST/t10k-images-idx3-ubyte.gz
Extracting data/MNIST/t10k-labels-idx1-ubyte.gz


In [3]:
print("-Training-set:\t\t{}".format(len(data.train.labels)))
print("-Test-set:\t\t{}".format(len(data.test.labels)))
print("-Validation-set:\t{}".format(len(data.validation.labels)))

-Training-set:		55000
-Test-set:		10000
-Validation-set:	5000


### Step 2. Build Model

In [4]:
sess = tf.InteractiveSession()

## Input and Ouput Layer-add placeholder
X = tf.placeholder(tf.float32, shape = [None, 784])
Y = tf.placeholder(tf.float32, shape = [None, 10])

In [5]:
## Filter Weight Definition
def filterWgt(shape, name):
    weights = tf.get_variable(name=name, shape=shape, 
                             initializer=tf.contrib.layers.xavier_initializer())
    return weights

## Bias Definition
def filterBias(shape, name):
    bias = tf.get_variable(name=name, shape=shape,
                          initializer=tf.zeros_initializer())
    
    return bias

## Conv Layer Definition
def conv2D(x, W):
    ## Stride: 1
    Conv = tf.nn.conv2d(x, W, strides=[1,1,1,1], padding="SAME")
    return Conv

## Avg_pooling Layer Definition
def avg_pool2D(x):
    ## Stride: 2, pooling kernel size:2*2
    return tf.nn.avg_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

## Max_pooling Layer Definition
def max_pool2D(x):
    ## Stride: 2, pooling kernel size: 2*2
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")


### Step 2.1. Build CNN Framework Sequential

In [6]:
## First Layer - Conv Layer

### reshape input dataset as 28*28 gray-scale image

W_conv1 = filterWgt([5,5,1,32], "weight1")
B_conv1 = filterBias([32], "bias1")

input_image = tf.reshape(X, [-1,28,28,1])

conv1_output = tf.nn.relu(conv2D(input_image, W_conv1) + B_conv1) ## 28*28 since padding=SAME

## Second Layer - Max Pooling
max_pool1_output = max_pool2D(conv1_output) ## 28-2+0/2+1 = 14*14

## Third Layer - Conv Layer
W_conv2 = filterWgt([5,5,32,64], "weight2")
B_conv2 = filterBias([64], "bias2")

conv2_output = tf.nn.relu(conv2D(max_pool1_output, W_conv2) + B_conv2) ## 14*14 since padding=SAME

## Forth Layer - Max Pooling
max_pool2_output = max_pool2D(conv2_output) ## 14-2+0/2+1 = 7*7

## Fifth Layer - Full connected Layer
W_fc1 = filterWgt([7*7*64, 1024], "weight3")
b_fc1 = filterBias([1024], "bias3")

### reshape max_pool2_output
max_pool2_flat = tf.reshape(max_pool2_output, [-1, 7*7*64])
fc1_output = tf.nn.relu(tf.matmul(max_pool2_flat, W_fc1) + b_fc1)

## Six Layer - subsample Full connected layer
W_fc2 = filterWgt([1024, 84], "weight4")
b_fc2 = filterBias([84], "bias4")
fc2_output = tf.nn.relu(tf.matmul(fc1_output, W_fc2) + b_fc2)

## Output Layer - Softmax
W_output = filterWgt([84, 10], "weight5")
b_output = filterBias([10], "bias5")
y_conv = tf.nn.softmax(tf.matmul(fc2_output, W_output) + b_output)

### Step 2.2 Define Loss function and Optimizer for Model 

In [7]:
## cross entropy - loss function
cross_entropy = -tf.reduce_sum(Y * tf.log(y_conv))

## optimizer
learning_rate = 1e-4
epoch = 10000

optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

## prediction
correct_predict = tf.equal(tf.argmax(y_conv, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, "float"))

### Step 3. Run training session

In [8]:
sess.run(tf.initialize_all_variables())

#start training, batch size is 50
for i in range(20000):
    batch = data.train.next_batch(50)
    if i%100 == 0:
        #每100次输出一次日志
        train_accuracy = accuracy.eval(feed_dict={
            X:batch[0], Y:batch[1]})
        print("step %d, training accuracy %g" % (i, train_accuracy))

    optimizer.run(feed_dict={X:batch[0], Y:batch[1]})

print("test accuracy %g" % accuracy.eval(feed_dict={X:data.test.images, Y:data.test.labels}))

    

W0104 06:48:03.439573 139975780869952 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/util/tf_should_use.py:193: initialize_all_variables (from tensorflow.python.ops.variables) is deprecated and will be removed after 2017-03-02.
Instructions for updating:
Use `tf.global_variables_initializer` instead.


step 0, training accuracy 0.04
step 100, training accuracy 0.8
step 200, training accuracy 0.9
step 300, training accuracy 0.96
step 400, training accuracy 0.98
step 500, training accuracy 0.92
step 600, training accuracy 0.96
step 700, training accuracy 0.96
step 800, training accuracy 0.98
step 900, training accuracy 0.96
step 1000, training accuracy 0.96
step 1100, training accuracy 1
step 1200, training accuracy 1
step 1300, training accuracy 0.98
step 1400, training accuracy 0.94
step 1500, training accuracy 0.92
step 1600, training accuracy 1
step 1700, training accuracy 0.98
step 1800, training accuracy 0.98
step 1900, training accuracy 0.98
step 2000, training accuracy 0.98
step 2100, training accuracy 1
step 2200, training accuracy 1
step 2300, training accuracy 1
step 2400, training accuracy 0.98
step 2500, training accuracy 0.98
step 2600, training accuracy 0.96
step 2700, training accuracy 1
step 2800, training accuracy 1
step 2900, training accuracy 0.98
step 3000, trainin